## Problem 2 and 3

The output for resnet running

<img src="Resnet.png" width="500" height="2000">

After changing the code to vgg this is the following result

<img src="VGG.png" width="500" height="2000">



## Problem 1

1. Data Preprocessing & Custom Channels

Custom Feature Engineering

    create_depth_abs_channels(image_tensor)

This function is critical. It calculates and adds two custom channels: one representing the absolute depth (a spatial linear gradient) and another representing image gradient magnitude (absolute difference of pixel intensity), improving the model's ability to locate boundaries.

Resizing

    X_resized = np.asarray(list(map(lambda x: cv2.resize(x, (224, 224)), X_train_ch)))

Resizes all images and masks to the standard 224×224 dimensions. This is necessary because ResNet50's pre-trained weights expect this input size.

2. Loss Function

Combined Loss

    def bce_dice_loss(y_true, y_pred): return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

This is the primary loss function, combining Binary Cross-Entropy (BCE), which handles pixel-wise classification, with the Dice Loss, which focuses on overlap and is better suited for handling heavy class imbalance (e.g., small salt masks)

3. Model Architecture

Encoder Backbone

    base_model = ResNet50(input_shape=input_size, include_top=False, weights=weights)
    
Uses the pre-trained ResNet50 without the final classification layers (include_top=False) as a powerful feature encoder, accelerating convergence.

U-Net Connections

    concat4 = concatenate([UpSampling2D()(decoder4), encoder4], axis=-1)

This is the essence of the U-Net architecture: it uses skip connections (concatenation) to merge high-level semantic features (from the decoder path) with high-resolution spatial details (from the encoder path).

Decoder Block

    def decoder_block_bottleneck(...)

Uses a bottleneck design within the decoder blocks to compress the feature maps before upsampling, potentially making the model more efficient and reducing redundancy.

Training Setup

Validation Split

    kfold = StratifiedKFold(n_splits=5, random_state=1337)

Uses Stratified K-Fold based on the custom coverage_class to ensure that each fold contains a representative distribution of salt mask sizes (coverages), leading to more reliable validation scores.

Best Model Saving

    ModelCheckpoint('unet_resnet.h5' ,monitor='val_my_iou_metric', mode='max', save_best_only=True, ...)

Only saves the model weights when the validation metric (val_my_iou_metric) is at its highest, ensuring the final output model is the best performing one.

Learning Rate Schedule

    reduce_lr = ReduceLROnPlateau(monitor='val_my_iou_metric', mode='max', factor=0.5, patience=5, ...)

Implements a dynamic learning rate schedule that automatically reduces the learning rate when the validation metric plateaus, aiding in final convergence and preventing overshooting the optimum.